In [ ]:
# Jorge's cell
import cv2
import matplotlib.pyplot as plt
import models
import numpy as np
import os
import torch
import utils.datasets
import utils.torch_utils
import utils.utils

In [ ]:
# Download COCO 2014 dataset (20GB):
!cwd=$(pwd) && cd .. && ${cwd}/data/get_coco_dataset_gdrive.sh && cd $cwd

In [ ]:
# Hyperparameters
hyp = {
    'giou': 3.31,   # giou loss gain
    'cls': 42.4,    # cls loss gain
    'cls_pw': 1.0,  # cls BCELoss positive_weight
    'obj': 52.0,    # obj loss gain (*=img_size/416 if img_size != 416)
    'obj_pw': 1.0,  # obj BCELoss positive_weight
    'iou_t': 0.213  # iou training threshold
}

# Other variables
architecture = "default"
cfg = "cfg/yolov3-spp.cfg"
data = "data/coco_64img.data"
path = "data/coco_64img.txt"
weights = "weights/ultralytics49.pt"
output_dir = "output/"
source_dir = "data/source/"
grad_filename = "grad.bmp"
original_filename = "original.jpg"
detect_filename = "detect.jpg"
number_of_classes = 80

# Initialize model
device = utils.torch_utils.select_device()
model = models.Darknet(cfg=cfg).to(device)
model.hyp = hyp               # attach hyperparameters to model
model.nc = number_of_classes  # attach number of classes to model
model.arc = architecture      # attach yolo architecture

# Load weights
models.attempt_download(weights)
if weights.endswith('.pt'):  # pytorch format
    model.load_state_dict(torch.load(weights, map_location=device)['model'])
else:  # darknet format
    _ = models.load_darknet_weights(model, weights)

# Dataset
dataset = utils.datasets.LoadImagesAndLabels(path)

def compute_saliency_map(index):
    # Compute gradient with respect to pixels
    img, target, img_path, _ = dataset[index]
    img = img.unsqueeze(0)
    img = img.to(device)
    target = target.to(device)
    img.requires_grad = True
    pred = model(img)    
    loss, _ = utils.utils.compute_loss(pred, target, model)
    loss.backward()
    img.requires_grad = False
    grad = img.grad
    img, grad = img.squeeze(), grad.squeeze()

    # Convert pixel gradients to grayscale image 
    grad = np.sum(np.absolute(grad.numpy()), axis=0)
    for i in range(1, 1000):
        max_idx = np.unravel_index(np.argmax(grad, axis=None), grad.shape)
        grad = grad / grad[max_idx] * 255
        print("average:", np.average(grad))
        if (np.average(grad) < 48):
            grad = grad / (192 * i)
            grad = np.log(grad + 1)
        else:
            break
    grad = grad.astype(int)

    # Plot pixel gradients
    plt.imshow(grad, cmap='Greys')
    plt.show()

    # Plot original image
    img = np.transpose(img.numpy(), (1, 2, 0))
    plt.imshow(img)
    plt.show()

    # Prepare environment
    !rm -rf $output_dir
    !mkdir $output_dir
    !rm -rf $source_dir
    !mkdir $source_dir
    !cp $img_path $source_dir

    # Detect bounding boxes
    !python3 detect.py --cfg $cfg --data $data --weights $weights --source $source_dir

    # Write pixel gradients to file
    cv2.imwrite(output_dir + grad_filename, grad)

    # Write original image to file
    img = img * 255
    img = img[:, :, [2, 1, 0]]  # Convert from RGB to BGR
    cv2.imwrite(output_dir + original_filename, img)

    # Plot original image with detected bounding boxes
    detect_img_path = output_dir + os.path.split(img_path)[1]
    img = cv2.imread(detect_img_path)
    img = img[:, :, [2, 1, 0]] # Convert from BGR to RGB
    img, _, _ = utils.datasets.letterbox(img=img, auto=False)
    plt.imshow(img)
    plt.show()

    # Write original image with detected bounding boxes to file
    img = img[:, :, [2, 1, 0]]  # Convert from RGB to BGR
    cv2.imwrite(output_dir + detect_filename, img)

In [ ]:
saliency_map_dir = "saliency_map_output/"

!rm -rf saliency_map_output
!mkdir saliency_map_output
for i in range(40):
    compute_saliency_map(i)
    path_full_size_detect = saliency_map_dir + "COCO" + str(i) + ".jpg"
    path_grad = saliency_map_dir + "grad" + str(i) + ".bmp"
    path_detect = saliency_map_dir + "detect" + str(i) + ".jpg"
    path_original = saliency_map_dir + "original" + str(i) + ".jpg"
    !cp output/COCO* $path_full_size_detect
    !cp output/grad.bmp $path_grad
    !cp output/detect.jpg $path_detect
    !cp output/original.jpg $path_original